In [1]:
import os
os.environ["OMP_NUM_THREADS"]="4"
os.environ["NCCL_IB_DISABLE"]="0"
os.environ["NCCL_IB_GID_INDEX"]="3"
os.environ["NCCL_SOCKET_IFNAME"]="bond0" #$(ifconfig | awk '/^[a-z]/ {gsub(/:/, ""); print $1; exit}')
os.environ["NCCL_DEBUG"]="INFO"

In [2]:
import ast
import os
import copy
from dataclasses import dataclass, field
import json
import logging
import pathlib
from typing import Dict, Optional, Sequence, List
from PIL import Image, ImageFile
from packaging import version
import numpy as np

import time
import random
import yaml
import math
import re
import torch

import transformers
import tokenizers
import deepspeed

from transformers import AutoConfig
from torch.utils.data import Dataset
from llava.constants import IGNORE_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN, IMAGE_TOKEN_INDEX
from llava.train.llava_trainer import LLaVATrainer

from llava import conversation as conversation_lib
from llava.model import *
from llava.mm_utils import process_highres_image, process_anyres_image, process_highres_image_crop_split, tokenizer_image_token
from llava.utils import rank0_print, process_video_with_pyav, process_video_with_decord
import torch.distributed as dist
torch.multiprocessing.set_sharing_strategy("file_system")

ImageFile.LOAD_TRUNCATED_IMAGES = True
local_rank = None

IS_TOKENIZER_GREATER_THAN_0_14 = version.parse(tokenizers.__version__) >= version.parse("0.14")

[2024-11-04 16:26:33,256] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/dpc/kunf0097/.conda/envs/llavanext/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import os

# os.environ["MASTER_ADDR"] = "172.18.10.4"
# os.environ["MASTER_PORT"] = "29544"
# dist.init_process_group(backend='nccl', rank=0, world_size=2, )

In [3]:
# local imports
from llava.train.train import *

In [4]:
import pickle
with open("parser.pkl", "rb") as f:
    p = pickle.load(f)
    model_args = p['model_args']
    data_args = p['data_args']
    training_args = p['training_args']

In [5]:
data_args.eval_data_path = "/home/kunet.ae/ku5001069/LLaVA-NeXT/data/s3/s3_test_v2.json"
# vars(data_args)

In [6]:
print("!!Inside train function!!")
global local_rank

# parser = transformers.HfArgumentParser((ModelArguments, DataArguments, TrainingArguments))
# model_args, data_args, training_args = parser.parse_args_into_dataclasses()

if training_args.verbose_logging:
    rank0_print(f"Inspecting experiment hyperparameters:\n")
    rank0_print(f"model_args = {vars(model_args)}\n\n")
    rank0_print(f"data_args = {vars(data_args)}\n\n")
    rank0_print(f"training_args = {vars(training_args)}\n\n")
    # rank0_print(f"evaluation_args = {vars(evaluation_args)}\n\n")

local_rank = training_args.local_rank
compute_dtype = torch.float16 if training_args.fp16 else (torch.bfloat16 if training_args.bf16 else torch.float32)

bnb_model_from_pretrained_args = {}
if training_args.bits in [4, 8]:
    from transformers import BitsAndBytesConfig

    bnb_model_from_pretrained_args.update(
        dict(
            device_map={"": training_args.device},
            load_in_4bit=training_args.bits == 4,
            load_in_8bit=training_args.bits == 8,
            quantization_config=BitsAndBytesConfig(
                load_in_4bit=training_args.bits == 4,
                load_in_8bit=training_args.bits == 8,
                llm_int8_threshold=6.0,
                llm_int8_has_fp16_weight=False,
                bnb_4bit_compute_dtype=compute_dtype,
                bnb_4bit_use_double_quant=training_args.double_quant,
                bnb_4bit_quant_type=training_args.quant_type,  # {'fp4', 'nf4'}
            ),
        )
    )

!!Inside train function!!
Inspecting experiment hyperparameters:

model_args = {'model_name_or_path': '/dpc/kunf0097/.cache/huggingface/hub/v2-llava-qwen-ov-s2-1028_182909', 'model_class_name': None, 'mm_tunable_parts': 'mm_vision_tower,mm_mlp_adapter,mm_language_model', 'version': 'qwen_1_5', 'freeze_backbone': False, 'tune_mm_mlp_adapter': False, 'tune_mm_vision_resampler': False, 'vision_tower': 'google/siglip-so400m-patch14-384', 'vision_tower_pretrained': None, 'unfreeze_mm_vision_tower': False, 'unfreeze_language_model': False, 'mm_vision_select_layer': -2, 'pretrain_mm_mlp_adapter': None, 'mm_projector_type': 'mlp2x_gelu', 'mm_use_im_start_end': False, 'mm_use_im_patch_token': False, 'mm_patch_merge_type': 'spatial_unpad', 'mm_vision_select_feature': 'patch', 'mm_resampler_type': None, 'mm_mask_drop_mode': 'fixed', 'mm_mask_drop_skip_percentage': 0.0, 'mm_mask_drop_ratio': 0.25, 'mm_mask_drop_ratio_upper': None, 'mm_mask_drop_ratio_lower': None, 'mm_spatial_pool_stride': None, '

In [ ]:
import evaluate
    metric = evaluate.load('bertscore')

    class BertScoreMetric:
        def __init__(self):
            self.precision_scores = []
            self.recall_scores = []
            self.f1_scores = []

        def update(self, predicted_deto, label_deto):
            # Calculate scores for this batch
            score = metric.compute(predictions=predicted_deto, references=label_deto, model_type="microsoft/deberta-xlarge-mnli")
            
            # Store batch-level scores
            self.precision_scores.extend(score['precision'])
            self.recall_scores.extend(score['recall'])
            self.f1_scores.extend(score['f1'])

        def compute(self):
            # Aggregate scores across batches
            mean = lambda l: sum(l) / len(l)
            result = {
                "precision": mean(self.precision_scores),
                "recall": mean(self.recall_scores),
                "f1": mean(self.f1_scores)
            }
            
            # Reset batch statistics
            self.precision_scores = []
            self.recall_scores = []
            self.f1_scores = []
            
            return result

    # Instantiate your custom metric
    bert_score_metric = BertScoreMetric()

    def compute_metrics(eval_pred, compute_result=True):
        rank0_print(f"Computing metrics... {compute_result}")
        
        # Extract predictions and labels
        predictions = eval_pred.predictions[0]  # Shape: (2, 201, 15200)
        predicted_classes = torch.argmax(predictions, dim=-1)  # Shape: (2, 201)
        labels = eval_pred.label_ids  # Shape: (2, 201)
        
        # Decode the predictions and labels
        predicted_deto = tokenizer.batch_decode(predicted_classes)
        label_deto = tokenizer.batch_decode(label[label > -1] for label in labels)
        
        # Update metric with this batch's decoded results
        bert_score_metric.update(predicted_deto, label_deto)
        
        if compute_result:
            # Aggregate and return final metrics
            return bert_score_metric.compute()

In [7]:
model = get_model(model_args, training_args, bnb_model_from_pretrained_args)
model.config.use_cache = False
if model_args.rope_scaling_factor is not None and model_args.rope_scaling_type is not None:
    model.config.rope_scaling = {
        "factor": model_args.rope_scaling_factor,
        "type": model_args.rope_scaling_type,
    }

if model_args.freeze_backbone:
    model.model.requires_grad_(False)

if training_args.bits in [4, 8]:
    from peft import prepare_model_for_kbit_training

    model.config.torch_dtype = torch.float32 if training_args.fp16 else (torch.bfloat16 if training_args.bf16 else torch.float32)
    model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=training_args.gradient_checkpointing)

if training_args.gradient_checkpointing:
    if hasattr(model, "enable_input_require_grads"):
        model.enable_input_require_grads()
    else:

        def make_inputs_require_grad(module, input, output):
            output.requires_grad_(True)

        model.get_input_embeddings().register_forward_hook(make_inputs_require_grad)

if training_args.lora_enable:
    from peft import LoraConfig, get_peft_model

    lora_config = LoraConfig(
        r=training_args.lora_r,
        lora_alpha=training_args.lora_alpha,
        target_modules=find_all_linear_names(model),
        lora_dropout=training_args.lora_dropout,
        bias=training_args.lora_bias,
        task_type="CAUSAL_LM",
    )
    if training_args.bits == 16:
        if training_args.bf16:
            model.to(torch.bfloat16)
        if training_args.fp16:
            model.to(torch.float16)
    rank0_print("Adding LoRA adapters...")
    model = get_peft_model(model, lora_config)

if "mistral" in model_args.model_name_or_path.lower() or "mixtral" in model_args.model_name_or_path.lower() or "zephyr" in model_args.model_name_or_path.lower():
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_args.model_name_or_path, cache_dir=training_args.cache_dir, model_max_length=training_args.model_max_length, padding_side="left")
elif "qwen" in model_args.model_name_or_path.lower():
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_args.model_name_or_path, cache_dir=training_args.cache_dir, model_max_length=training_args.model_max_length, padding_side="right")
elif (
    "wizardlm-2" in model_args.model_name_or_path.lower()
    or "vicuna" in model_args.model_name_or_path.lower()
    or "llama" in model_args.model_name_or_path.lower()
    or "yi" in model_args.model_name_or_path.lower()
    or "nous-hermes" in model_args.model_name_or_path.lower()
    and "wizard-2" in model_args.model_name_or_path.lower()
):
    tokenizer = transformers.AutoTokenizer.from_pretrained(
        model_args.model_name_or_path,
        cache_dir=training_args.cache_dir,
        model_max_length=training_args.model_max_length,
        padding_side="right",
        use_fast=False,
    )

rank0_print(f"Prompt version: {model_args.version}")
if model_args.version == "v0":
    if tokenizer.pad_token is None:
        smart_tokenizer_and_embedding_resize(
            special_tokens_dict=dict(pad_token="[PAD]"),
            tokenizer=tokenizer,
            model=model,
        )
elif model_args.version == "v0.5":
    tokenizer.pad_token = tokenizer.unk_token
else:
    if tokenizer.unk_token is not None:
        tokenizer.pad_token = tokenizer.unk_token
    if model_args.version in conversation_lib.conv_templates:
        conversation_lib.default_conversation = conversation_lib.conv_templates[model_args.version]
    else:
        conversation_lib.default_conversation = conversation_lib.conv_templates["vicuna_v1"]


Overwriting config with {'use_pos_skipping': False, 'pos_skipping_range': 4096, 'mm_spatial_pool_mode': 'bilinear'}
Loading vision tower: google/siglip-so400m-patch14-384
Adding LoRA adapters...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Prompt version: qwen_1_5


In [8]:
if model_args.vision_tower is not None:
    model.get_model().initialize_vision_modules(model_args=model_args, fsdp=training_args.fsdp)

    vision_tower = model.get_vision_tower()
    vision_tower.to(dtype=torch.bfloat16 if training_args.bf16 else torch.float16, device=training_args.device)

    data_args.image_processor = vision_tower.image_processor
    data_args.is_multimodal = True

    model.config.image_aspect_ratio = data_args.image_aspect_ratio
    if data_args.image_grid_pinpoints is not None:
        if isinstance(data_args.image_grid_pinpoints, str) and "x" in data_args.image_grid_pinpoints:
            try:
                patch_size = data_args.image_processor.size[0]
            except Exception as e:
                patch_size = data_args.image_processor.size["shortest_edge"]

            assert patch_size in [224, 336, 384, 448, 512], "patch_size should be in [224, 336, 384, 448, 512]"
            # Use regex to extract the range from the input string
            matches = re.findall(r"\((\d+)x(\d+)\)", data_args.image_grid_pinpoints)
            range_start = tuple(map(int, matches[0]))
            range_end = tuple(map(int, matches[-1]))
            # Generate a matrix of tuples from (range_start[0], range_start[1]) to (range_end[0], range_end[1])
            grid_pinpoints = [(i, j) for i in range(range_start[0], range_end[0] + 1) for j in range(range_start[1], range_end[1] + 1)]
            # Multiply all elements by patch_size
            data_args.image_grid_pinpoints = [[dim * patch_size for dim in pair] for pair in grid_pinpoints]
        elif isinstance(data_args.image_grid_pinpoints, str):
            data_args.image_grid_pinpoints = ast.literal_eval(data_args.image_grid_pinpoints)

    model.config.image_grid_pinpoints = data_args.image_grid_pinpoints
    model.config.image_crop_resolution = data_args.image_crop_resolution
    model.config.image_split_resolution = data_args.image_split_resolution
    model.config.tokenizer_padding_side = tokenizer.padding_side
    model.config.tokenizer_model_max_length = tokenizer.model_max_length
    model.config.mm_newline_position = model_args.mm_newline_position
    model.config.add_faster_video = model_args.add_faster_video
    model.config.faster_token_stride = model_args.faster_token_stride
    model.config.add_time_instruction = data_args.add_time_instruction
    model.config.force_sample = data_args.force_sample
    model.config.mm_spatial_pool_stride = model_args.mm_spatial_pool_stride 

    ### Deciding train which part of the model
    if model_args.mm_tunable_parts is None:  # traditional way of deciding which part to train
        model.config.tune_mm_mlp_adapter = training_args.tune_mm_mlp_adapter = model_args.tune_mm_mlp_adapter
        model.config.tune_mm_vision_resampler = training_args.tune_mm_vision_resampler = model_args.tune_mm_vision_resampler
        if model_args.tune_mm_mlp_adapter or model_args.tune_mm_vision_resampler:
            model.requires_grad_(False)
        if model_args.tune_mm_mlp_adapter:
            for p in model.get_model().mm_projector.parameters():
                p.requires_grad = True
        if model_args.tune_mm_vision_resampler:
            for p in model.get_model().vision_resampler.parameters():
                p.requires_grad = True

        model.config.freeze_mm_mlp_adapter = training_args.freeze_mm_mlp_adapter
        if training_args.freeze_mm_mlp_adapter:
            for p in model.get_model().mm_projector.parameters():
                p.requires_grad = False

        model.config.freeze_mm_vision_resampler = training_args.freeze_mm_vision_resampler
        if training_args.freeze_mm_vision_resampler:
            for p in model.get_model().vision_resampler.parameters():
                p.requires_grad = False

        model.config.unfreeze_mm_vision_tower = model_args.unfreeze_mm_vision_tower
        if model_args.unfreeze_mm_vision_tower:
            vision_tower.requires_grad_(True)
        else:
            vision_tower.requires_grad_(False)

    else:
        rank0_print(f"Using mm_tunable_parts: {model_args.mm_tunable_parts}")
        model.config.mm_tunable_parts = training_args.mm_tunable_parts = model_args.mm_tunable_parts
        # Set the entire model to not require gradients by default
        model.requires_grad_(False)
        vision_tower.requires_grad_(False)
        model.get_model().mm_projector.requires_grad_(False)
        model.get_model().vision_resampler.requires_grad_(False)
        # Parse the mm_tunable_parts to decide which parts to unfreeze
        tunable_parts = model_args.mm_tunable_parts.split(",")
        if "mm_mlp_adapter" in tunable_parts:
            for p in model.get_model().mm_projector.parameters():
                p.requires_grad = True
        if "mm_vision_resampler" in tunable_parts:
            for p in model.get_model().vision_resampler.parameters():
                p.requires_grad = True
        if "mm_vision_tower" in tunable_parts:
            for name, param in model.named_parameters():
                if "vision_tower" in name:
                    param.requires_grad_(True)
        if "mm_language_model" in tunable_parts:
            for name, param in model.named_parameters():
                if "vision_tower" not in name and "mm_projector" not in name and "vision_resampler" not in name:
                    param.requires_grad_(True)

    total_params = sum(p.ds_numel if hasattr(p, "ds_numel") else p.numel() for p in model.parameters())
    trainable_params = sum(p.ds_numel if hasattr(p, "ds_numel") else p.numel() for p in model.parameters() if p.requires_grad)
    rank0_print(f"Total parameters: ~{total_params/1e6:.2f} MB)")
    rank0_print(f"Trainable parameters: ~{trainable_params/1e6:.2f} MB)")
    if training_args.bits in [4, 8]:
        model.get_model().mm_projector.to(dtype=compute_dtype, device=training_args.device)

    model.config.mm_use_im_start_end = data_args.mm_use_im_start_end = model_args.mm_use_im_start_end
    model.config.mm_projector_lr = training_args.mm_projector_lr
    model.config.mm_vision_tower_lr = training_args.mm_vision_tower_lr
    training_args.use_im_start_end = model_args.mm_use_im_start_end
    model.config.mm_use_im_patch_token = model_args.mm_use_im_patch_token
    model.initialize_vision_tokenizer(model_args, tokenizer=tokenizer)

google/siglip-so400m-patch14-384 is already loaded, `load_model` called again, skipping.
Using mm_tunable_parts: mm_vision_tower,mm_mlp_adapter,mm_language_model
Total parameters: ~1040.91 MB)
Trainable parameters: ~1040.91 MB)


In [9]:
if training_args.bits in [4, 8]:
    from peft.tuners.lora import LoraLayer

    for name, module in model.named_modules():
        if isinstance(module, LoraLayer):
            if training_args.bf16:
                module = module.to(torch.bfloat16)
        if "norm" in name:
            module = module.to(torch.float32)
        if "lm_head" in name or "embed_tokens" in name:
            if hasattr(module, "weight"):
                if training_args.bf16 and module.weight.dtype == torch.float32:
                    module = module.to(torch.bfloat16)


In [10]:
data_module = make_supervised_data_module(tokenizer=tokenizer, data_args=data_args)

Loading /home/kunet.ae/ku5001069/LLaVA-NeXT/data/s3/s3_train_v2.json
Loaded 1525 samples from /home/kunet.ae/ku5001069/LLaVA-NeXT/data/s3/s3_train_v2.json
Loaded 1525 samples from /home/kunet.ae/ku5001069/LLaVA-NeXT/data/s3/s3_train_v2.json
Formatting inputs...Skip in lazy mode
Loading /home/kunet.ae/ku5001069/LLaVA-NeXT/data/s3/s3_test_v2.json
Loaded 382 samples from /home/kunet.ae/ku5001069/LLaVA-NeXT/data/s3/s3_test_v2.json
Loaded 382 samples from /home/kunet.ae/ku5001069/LLaVA-NeXT/data/s3/s3_test_v2.json
Formatting inputs...Skip in lazy mode


In [11]:
import evaluate
metric = evaluate.load('bertscore')
def compute_metrics(eval_pred, compute_result=True):  
    predictions = eval_pred.predictions[0]  # Shape: (2, 201, 15200)
    
    predicted_classes = torch.argmax(predictions, dim=-1)  # Shape: (2, 201)
    labels = eval_pred.label_ids             # Shape: (2, 201)
    
    predicted_deto = tokenizer.batch_decode(predicted_classes)
    label_deto = tokenizer.batch_decode(label[label > -1] for label in labels)
    
    print(predicted_deto)
    print(label_deto)
    
    score = metric.compute(predictions=predicted_deto, references=label_deto, model_type="microsoft/deberta-xlarge-mnli")
    mean = lambda l: sum(l) / len(l)
    print(score)
    # a = 2/0
    return {'precision': mean(score['precision']), 'recall': mean(score['recall']), 'f1': mean(score['f1'])}

training_args.report_to = []

trainer = LLaVATrainer(model=model, 
                        tokenizer=tokenizer, 
                        args=training_args,
                        compute_metrics=compute_metrics, 
                        **data_module)
torch.cuda.empty_cache()

if list(pathlib.Path(training_args.output_dir).glob("checkpoint-*")):
    trainer.train(resume_from_checkpoint=True)
else:
    trainer.train()

/dpc/kunf0097/.conda/envs/llavanext/lib/python3.10/site-packages/accelerate/accelerator.py:457: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=None)
  warnings.warn(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Setting NCCL timeout to INF to avoid running errors.


Step,Training Loss,Validation Loss


RuntimeError: Default process group has not been initialized, please make sure to call init_process_group.

In [ ]:
trainer.save_state()

model.config.use_cache = True

if training_args.lora_enable:
    state_dict = get_peft_state_maybe_zero_3(model.named_parameters(), training_args.lora_bias)
    non_lora_state_dict = get_peft_state_non_lora_maybe_zero_3(model.named_parameters())
    if training_args.local_rank == 0 or training_args.local_rank == -1:
        if hasattr(model, "config"):
            model.config.save_pretrained(training_args.output_dir)
        if hasattr(model, "generation_config"):
            model.generation_config.save_pretrained(training_args.output_dir)
        model.save_pretrained(training_args.output_dir, state_dict=state_dict)
        torch.save(non_lora_state_dict, os.path.join(training_args.output_dir, "non_lora_trainables.bin"))
else:
    safe_save_model_for_hf_trainer(trainer=trainer, output_dir=training_args.output_dir)

rank0_print(f"Model saved to {training_args.output_dir}")